In [2]:
import pandas as pd
import numpy as np


In [3]:
movies = pd.read_csv('../data/tmdb_5000_movies.csv')


In [ ]:
credits = pd.read_csv('../data/tmdb_5000_credits.csv')


In [9]:
movies = movies.merge(credits,on='title')

In [10]:
movies.shape


(4809, 23)

In [12]:
movies = movies[['genres','id','keywords','overview','title','cast','crew']]

In [14]:
movies.isnull().sum()

genres      0
id          0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [15]:
movies.dropna(inplace=True)

In [16]:
movies.duplicated().sum()

0

In [17]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj) :
        l.append(i['name'])
    return l


In [19]:
import ast

In [20]:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [21]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [22]:
movies['genres'] = movies['genres'].apply(convert)

In [24]:
movies['keywords'] = movies['keywords'].apply(convert)

In [25]:
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [26]:
def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(i['name'])
            counter +=1
        else:
            break
    return l

In [27]:
movies['cast'] = movies['cast'].apply(convert2)

In [29]:
def fetch_dir(obj):
    l= []
    for i in ast.literal_eval(obj):
        if i['job'] == "Director" :
            l.append(i['name'])
            break
    return l
        

In [30]:
movies['crew']  = movies['crew'].apply(fetch_dir)

In [32]:
movies['overview'] = movies['overview'].apply(lambda i : i.split())

In [34]:
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [36]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [38]:
new_df = movies[['id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

In [41]:
new_df.head()

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [42]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

C:\Users\JAYDEEP\AppData\Local\Temp\ipykernel_13936\3679624391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [46]:
vector  = cv.fit_transform(new_df['tags']).toarray()

In [48]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [50]:
from nltk.stem.porter import PorterStemmer

In [51]:
ps  = PorterStemmer()

In [52]:
def stem(text) :
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [53]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\JAYDEEP\AppData\Local\Temp\ipykernel_13936\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
similarity =  cosine_similarity(vector)

In [57]:
sorted(list(enumerate(similarity[0])),reverse = True,key = lambda x : x[1])[1:9]


[(539, 0.26089696604360174),
 (1192, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1214, 0.24944382578492943),
 (582, 0.24397501823713333),
 (1916, 0.243599382882345),
 (3728, 0.23904572186687872)]

In [58]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    dist = similarity[index]
    movies_list = sorted(list(enumerate(dist)),reverse = True,key = lambda x : x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)



In [59]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [60]:
recommend('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns


In [61]:
recommend('Superman')

Superman II
Superman Returns
Superman III
Superman IV: The Quest for Peace
Man of Steel


In [62]:
recommend('Interstellar')

Silent Running
Space Pirate Captain Harlock
The Martian
Apollo 13
Moonraker


In [63]:
import pickle

pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [64]:
pickle.dump(similarity,open('similarity.pkl','wb'))